<a href="https://colab.research.google.com/github/bankehsaz/Holt-Winters-vs-Naive/blob/main/10_Holt_Winters_Model_for_Trading_Gold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import packages
import numpy as np
import pandas as pd
import plotly.express as px
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import yfinance as yf
from sklearn import metrics

In [2]:
# Get data from yahoo finance
df = yf.download('GLD', start='2022-01-01', end='2024-01-01')
df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-06-26,179.050003,179.080002,178.419998,178.509995,178.509995,3222100
2023-06-27,179.070007,179.289993,177.429993,177.690002,177.690002,5036900
2023-06-28,177.229996,177.690002,176.699997,177.279999,177.279999,5866700
2023-06-29,175.830002,177.630005,175.789993,177.089996,177.089996,6862500
2023-06-30,177.690002,178.529999,177.320007,178.270004,178.270004,6420600


In [3]:
# plot
px.line(df, x=df.index, y='Close')

In [4]:
# Log Transform
df['Log-Close'] = np.log(df['Close'])

In [5]:
# plot
px.line(df, x=df.index, y='Log-Close')

In [6]:
# Split data into train and test
n_test = 30
train = df.iloc[: -n_test]
test = df.iloc[-n_test:]

In [7]:
# Set frequency to 'business day' not working
# df.index.freq = 'B'

In [8]:
# Instantiation
model = ExponentialSmoothing(train['Log-Close'], trend='add', seasonal=None, initialization_method='legacy-heuristic')

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [9]:
# Fit the model
res = model.fit()

In [10]:
# Set index for 'Holt-Winters' Column
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

In [11]:
# Calculate Train value and Forecast value for Holt-Winters Column
df.loc[train_idx, 'Holt-Winters-Train'] = res.fittedvalues
df.loc[test_idx, 'Holt-Winters-Test'] = res.forecast(n_test).to_numpy()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



In [12]:
# Plot
px.line(df, x=df.index, y=['Log-Close', 'Holt-Winters-Train', 'Holt-Winters-Test'])

In [13]:
y_true_Holt = df.iloc[-n_test:]['Log-Close']
y_pred_Holt = res.forecast(n_test)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



## Compare Holt-Winters Model with Naive Model

In [14]:
# close prediction
df['Log_Close_Prediction'] = df['Log-Close'].shift(1)
df.head()

,Open,High,Low,Close,Adj Close,Volume,Log-Close,Holt-Winters-Train,Holt-Winters-Test,Log_Close_Prediction
Date,,,,,,,,,,
2022-01-03,168.860001,169.009995,168.000000,168.330002,168.330002,9014400,5.125926,5.125996,NaN,NaN
2022-01-04,168.899994,169.720001,168.729996,169.570007,169.570007,6965600,5.133266,5.126190,NaN,5.125926
2022-01-05,170.619995,170.929993,168.899994,169.059998,169.059998,8715600,5.130254,5.133456,NaN,5.133266
2022-01-06,167.160004,167.750000,166.860001,166.990005,166.990005,10902700,5.117934,5.130549,NaN,5.130254
2022-01-07,167.369995,168.009995,166.860001,167.750000,167.750000,8191900,5.122475,5.118326,NaN,5.117934


In [15]:
# assign values to y_true and y_pred
# df.iloc[1:] ---> Because first row is NaN
y_true_Naive = df.iloc[1:]['Log-Close']
y_pred_Naive = df.iloc[1:]['Log_Close_Prediction']

In [16]:
# The Mean absolute percentage error(MAPE)
MAPE_Naive = metrics.mean_absolute_percentage_error(y_true_Naive, y_pred_Naive)
MAPE_Holt = metrics.mean_absolute_percentage_error(y_true_Holt, y_pred_Holt)
print('Naive MAPE: ', MAPE_Naive)
print('Holt MAPE: ', MAPE_Holt)

Naive MAPE:  0.0014188653227571403
Holt MAPE:  0.0044185200008290204


In [17]:
# The R^2
R2_Naive = metrics.r2_score(y_true_Naive, y_pred_Naive)
R2_Holt = metrics.r2_score(y_true_Holt, y_pred_Holt)
print('Naive R2: ', R2_Naive)
print('Holt R2: ', R2_Holt)

Naive R2:  0.9724038907836284
Holt R2:  -4.8438333160915805
